In [ ]:
import pymssql
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
server = 'localhost'
user = 'pythonuser'
password = input('Supply DB Password: ')
database = 'CaseStudy'

try:
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor(as_dict=True)

    cursor.callproc('mlb.getTrainingData_MultiHit')
    cursor.nextset()
    results = cursor.fetchall()
    training_data = pd.DataFrame(results) 
except Exception as e:
    print ('Exception',str(e))
finally:
    conn.close()
    

In [ ]:
# verify the size of the training data set
training_data.shape


In [ ]:
# removing individual descriptive columns, we want to ensure we do not train on
training_data = training_data.drop(['gameId','Date','batter_id','pitcher_id'], axis=1)


In [ ]:
# create training and testing splits
df_train, df_test = train_test_split(training_data, test_size=0.1, random_state=42)


In [ ]:
y_train = df_train['outcome'].values
y_test = df_test['outcome'].values
X_train = df_train.drop('outcome', axis=1).values
X_test = df_test.drop('outcome', axis=1).values


In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

print('Start training...')

gbm = lgb.LGBMClassifier()

gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

print('Done Training.')

In [ ]:
from sklearn.externals import joblib
# save model
joblib.dump(gbm, 'LGBMClassifier_Train_New_MultiHit.pkl')
# load model
gbm = joblib.load('LGBMClassifier_Train_New_MultiHit.pkl')

In [ ]:
results = pd.DataFrame( X_test.copy() )
predicted_probabilities = gbm.predict_proba(results)
results['predicted_probabilities'] = [el[1] for el in predicted_probabilities]
results['Success'] = y_test


In [ ]:
# later for serving the model

try:
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor(as_dict=True)

    cursor.callproc('mlb.getTodayStatsForNewTrainedModel')
    cursor.nextset()
    results = cursor.fetchall()
    today = pd.DataFrame(results,columns=['Batter', 'gameId', 'Date', 'batter_id', 'pitcher_id', 'BatterAvgHitsPerGameLast104', 'BatterAvgHitsPerGameLast1800', 'BatterAvgHitsPerGameThisSeason', 'BatterAvgHitsPerGame_InLast10Quarters', 'BatterAvgHitsPerGame_InLast11Quarters', 'BatterAvgHitsPerGame_InLast12Quarters', 'BatterAvgHitsPerGame_InLast13Quarters', 'BatterAvgHitsPerGame_InLast15Quarters', 'BatterAvgHitsPerGame_InLast16Quarters', 'BatterAvgHitsPerGame_InLast17Quarters', 'BatterAvgHitsPerGame_InLast18Quarters', 'BatterAvgHitsPerGame_InLast19Quarters', 'BatterAvgHitsPerGame_InLast20Quarters', 'BatterAvgHitsPerGame_InLast3Quarters', 'BatterAvgHitsPerGame_InLast4Quarters', 'BatterAvgHitsPerGame_InLast5Quarters', 'BatterAvgHitsPerGame_InLast6Quarters', 'BatterAvgHitsPerGame_InLast7Quarters', 'BatterAvgHitsPerGame_InLast8Quarters', 'BatterAvgHitsPerGame_InLast9Quarters', 'BatterGamesInLast104', 'BatterGamesInLast1800', 'BatterGamesInLast42Days', 'BatterGamesInLast708Days', 'BatterGamesThisSeason', 'BatterGwH_InLast9Quarters', 'BatterLast42Hits', 'BatterTimeOfDayAvgLast999', 'HistAB', 'OppAvg_InLast16Quarters', 'OppAvg_InLast4Quarters', 'OppAvg_InLast5Quarters', 'OppAvg_InLast6Quarters', 'OppAvg_InLast7Quarters', 'PitcherGamesStartedInLast568', 'PitcherGames_InLast4Quarters', 'PitcherOppAvg568', 'StadiumAVG_InLast11Quarters', 'StadiumAVG_InLast1Quarters', 'StadiumAVG_InLast2Quarters', 'StadiumAVG_InLast3Quarters', 'StadiumAVG_InLast4Quarters', 'StadiumAVG_InLast5Quarters', 'StadiumAVG_InLast9Quarters', 'StadiumGwH602', 'ab_InLast20Day', 'ab_InLast6Day', 'avgHitsPerGameStarted_InLast16Quarters', 'avg_InLast10Day', 'bAvg_InLast18Quarters', 'bAvg_InLast8Quarters', 'bGamesWithAHit708', 'batterTeamAvgLast924', 'h_InLast10Day', 'h_InLast22Day', 'h_InLast24Day', 'h_InLast3Day', 'pAvgVsHandedBatter_InLast3Quarters', 'pAvgVsHandedBatter_InLast4Quarters', 'pHits_InLast8Quarters', 'pitcherTeamAvgLast761', 'pitcherTeamAvg_InLast1Quarters', 'pitcherTeamAvg_InLast2Quarters', 'pitcherTeamAvg_InLast3Quarters', 'pitcherTeamAvg_InLast6Quarters', 'tpa_InLast19Day', 'tpa_InLast1Day']) 
except Exception as e:
    print ('Exception',str(e))
finally:
    conn.close()
  
todayForModel = today.drop(['Batter', 'gameId', 'Date', 'batter_id', 'pitcher_id'], axis=1)

# confirm columns order is as expected
assert list(todayForModel) == ['BatterAvgHitsPerGameLast104', 'BatterAvgHitsPerGameLast1800', 'BatterAvgHitsPerGameThisSeason', 'BatterAvgHitsPerGame_InLast10Quarters', 'BatterAvgHitsPerGame_InLast11Quarters', 'BatterAvgHitsPerGame_InLast12Quarters', 'BatterAvgHitsPerGame_InLast13Quarters', 'BatterAvgHitsPerGame_InLast15Quarters', 'BatterAvgHitsPerGame_InLast16Quarters', 'BatterAvgHitsPerGame_InLast17Quarters', 'BatterAvgHitsPerGame_InLast18Quarters', 'BatterAvgHitsPerGame_InLast19Quarters', 'BatterAvgHitsPerGame_InLast20Quarters', 'BatterAvgHitsPerGame_InLast3Quarters', 'BatterAvgHitsPerGame_InLast4Quarters', 'BatterAvgHitsPerGame_InLast5Quarters', 'BatterAvgHitsPerGame_InLast6Quarters', 'BatterAvgHitsPerGame_InLast7Quarters', 'BatterAvgHitsPerGame_InLast8Quarters', 'BatterAvgHitsPerGame_InLast9Quarters', 'BatterGamesInLast104', 'BatterGamesInLast1800', 'BatterGamesInLast42Days', 'BatterGamesInLast708Days', 'BatterGamesThisSeason', 'BatterGwH_InLast9Quarters', 'BatterLast42Hits', 'BatterTimeOfDayAvgLast999', 'HistAB', 'OppAvg_InLast16Quarters', 'OppAvg_InLast4Quarters', 'OppAvg_InLast5Quarters', 'OppAvg_InLast6Quarters', 'OppAvg_InLast7Quarters', 'PitcherGamesStartedInLast568', 'PitcherGames_InLast4Quarters', 'PitcherOppAvg568', 'StadiumAVG_InLast11Quarters', 'StadiumAVG_InLast1Quarters', 'StadiumAVG_InLast2Quarters', 'StadiumAVG_InLast3Quarters', 'StadiumAVG_InLast4Quarters', 'StadiumAVG_InLast5Quarters', 'StadiumAVG_InLast9Quarters', 'StadiumGwH602', 'ab_InLast20Day', 'ab_InLast6Day', 'avgHitsPerGameStarted_InLast16Quarters', 'avg_InLast10Day', 'bAvg_InLast18Quarters', 'bAvg_InLast8Quarters', 'bGamesWithAHit708', 'batterTeamAvgLast924', 'h_InLast10Day', 'h_InLast22Day', 'h_InLast24Day', 'h_InLast3Day', 'pAvgVsHandedBatter_InLast3Quarters', 'pAvgVsHandedBatter_InLast4Quarters', 'pHits_InLast8Quarters', 'pitcherTeamAvgLast761', 'pitcherTeamAvg_InLast1Quarters', 'pitcherTeamAvg_InLast2Quarters', 'pitcherTeamAvg_InLast3Quarters', 'pitcherTeamAvg_InLast6Quarters', 'tpa_InLast19Day', 'tpa_InLast1Day']

preds_proba = gbm.predict_proba( todayForModel )      

today['predicted_probabilities'] = [el[1] for el in preds_proba]
today = today.sort_values(['predicted_probabilities'], ascending=[0])

today[['Date','Batter','predicted_probabilities']]
